In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
items=pd.read_excel(r"data\\revidly.xlsx",sheet_name='train')

In [3]:
items.head()
items.to_excel('data\\revidly.xlsx',sheet_name='train',index=False)


In [4]:
n_users=items.user_id.unique().shape[0]
n_users

301

In [5]:
n_items=items.post_id.unique().shape[0]
n_items

1001

In [6]:
items.describe()

,user_id,post_id,vote,time_spent,share,n_views,time_scaled,score
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.00000,12000.000000,12000.000000,12000.000000
mean,149.330833,501.794833,0.004333,149.736417,24.90775,246.939667,0.499121,0.251107
std,87.162114,289.558051,0.818456,86.860013,14.70490,144.517073,0.289533,0.421783
min,0.000000,0.000000,-1.000000,0.000000,0.00000,0.000000,0.000000,-0.489400
25%,74.000000,252.000000,-1.000000,75.000000,12.00000,122.000000,0.250000,-0.177650
50%,150.000000,505.000000,0.000000,149.000000,25.00000,246.000000,0.496667,0.252200
75%,225.000000,753.000000,1.000000,225.000000,38.00000,371.000000,0.750000,0.675400
max,300.000000,1000.000000,1.000000,300.000000,50.00000,500.000000,1.000000,0.985400


In [7]:
items['time_spent'].mean()

149.73641666666666

In [8]:
#normalize time_Spent
#items['time']=(items['time_spent'])/300 #-items['time_spent'].mean())/150


In [9]:
#normalize vote
#items.vote=items.vote/2

In [10]:
#define new rating
score= (items['vote'] * 0.5 ) + (items['time_scaled'] *0.3) +(items['share']*0.1/50)+(items['n_views']*0.1/500)


In [11]:
items['score']=score
items.head(20)

,user_id,post_id,vote,time_spent,share,n_views,time_scaled,score
0,191,10,0,153,12,410,0.510000,0.2590
1,121,697,0,66,37,318,0.220000,0.2036
2,267,680,0,41,31,121,0.136667,0.1272
3,15,584,0,20,9,118,0.066667,0.0616
4,194,996,-1,226,8,264,0.753333,-0.2052
5,72,827,1,62,10,242,0.206667,0.6304
6,24,446,1,268,19,267,0.893333,0.8594
7,146,345,-1,193,50,439,0.643333,-0.1192
8,11,61,1,2,43,180,0.006667,0.6240
9,208,167,0,63,24,74,0.210000,0.1258


In [12]:
#data_matrix is the user x item matrix, which has rating of user i for the post j

data_matrix = np.zeros((n_users, n_items))#,2))


In [13]:
# data matrix . rows are users and columns are movies. the value at data_matrix[i][j]=rating for movie j by user i.
for line in items.itertuples():
    #print ([line[3],line[4]])
    data_matrix[line[1],line[2]]=line[8]  #,line[4]]

In [14]:
data_matrix[191][10]

0.259

In [15]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine') 
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [16]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [17]:
#user_prediction = predict(data_matrix, user_similarity, type='user') 
item_prediction = predict(data_matrix, item_similarity, type='item')

In [18]:
item_prediction.argsort()

array([[115, 875, 165, ..., 411,  62, 552],
       [328, 154, 204, ..., 137, 637,  33],
       [143, 185, 307, ..., 883, 114, 158],
       ...,
       [ 58,  66, 203, ..., 492, 921, 977],
       [ 34, 704, 847, ..., 220, 536, 698],
       [ 27, 212, 622, ...,  54,  83, 643]], dtype=int64)

In [19]:
#n_post= number of post want to suggest to each user
n_post=20

pred_post=np.argsort(-item_prediction)[:,:n_post]
pred_post[0]


array([552,  62, 411, 507, 571, 131, 343, 374, 899, 597, 716, 101,  92,
       654, 858, 645, 538, 225, 510, 105], dtype=int64)

In [20]:
pred_item=pd.DataFrame({'user_id':[] , 'post_ids':[]})

for i in range(n_users):
    pred_item = pred_item.append({'user_id': i, 'post_ids':pred_post[i]}, ignore_index=True)
    #pred_item['post_ids'][i]=pred_post[i]
    
pred_item.user_id.astype='int'

In [21]:
pred_item.head()

,user_id,post_ids
0,0.0,"[552, 62, 411, 507, 571, 131, 343, 374, 899, 5..."
1,1.0,"[33, 637, 137, 523, 822, 776, 26, 422, 911, 97..."
2,2.0,"[158, 114, 883, 683, 159, 755, 864, 81, 929, 2..."
3,3.0,"[71, 108, 626, 534, 939, 600, 988, 444, 324, 8..."
4,4.0,"[124, 452, 879, 573, 438, 905, 505, 65, 120, 9..."


In [22]:
# from openpyxl import load_workbook
# workbook1 = load_workbook('data\\revidly.xlsx')
 
# writer = pd.ExcelWriter('data\\revidly.xlsx', engine='openpyxl') 
# writer.book = workbook1
# pred_item.to_excel(writer, sheet_name='pred_output',index=False)
 
# writer.save()
# writer.close()